In [10]:
import numpy as np
from scipy import integrate
import time
import pandas as pd
import sys # To run the batch script
import os
import sys

# Functions defined here

In [53]:
def find_indices_of_wavelengths(radiationField_wavelengths_filename: str, 
                                min_wavelength: float, 
                                max_wavelength: float) -> np.ndarray:
    
    """
    Returns indices of wavelengths.
    
    Parameters:
    -----------
    radiationField_wavelengths_filename : str
        Path to the file containing wavelength data.
        
    min_wavelength: float
        Smallest wavelength of interest.
        micron

    max_wavelength: float
        Largest wavelength of interest.
        micron

    Returns:
    --------
    indices:
        Indices corresponding to wavelengths.
        Note: The first column of the `radiationField_J.dat` file represents the cell index number. 
        Thus, when using these indices on that file, remember to increment each index by 1.
        
    wavelengths_filtered: 
        Center wavelengths in each measurement. 

    Example:
    --------
    >>> indices, wavelengths = find_indices_of_wavelengths_corresponding_to_the_habing_radiation_field("radiationField_wavelengths.dat")
    >>> print(indices)
    [index1, index2, ...]
    
    """
    
    
    print("I am in the function find_indices_of_wavelengths")
        
    print(f"{radiationField_wavelengths_filename} is started to read.")
    wavelengths_content = np.genfromtxt(radiationField_wavelengths_filename, delimiter = ' ', skip_header = 1)
    print(f"{radiationField_wavelengths_filename} was read.")
    
    wavelengths = wavelengths_content[:,0]
    
    indices = np.where((wavelengths < max_wavelength) & (wavelengths > min_wavelength))[0]
    wavelengths_filtered = wavelengths[indices] 
    
    return indices, wavelengths_filtered


def flux_calculator(
    cell_content, 
    wavelengths,
):
    specific_intensities = cell_content[1:]
    # Integration: https://en.wikipedia.org/wiki/Simpson%27s_rule
    intensities = integrate.simpson(y=specific_intensities, x=wavelengths)  # W/m^2/sr
    # Flux 
    flux = 4 * np.pi * intensities # W/m^2
    return flux

In [7]:
galaxy_name = "gal200"
which_FIRE = "firebox"
redshift = "0.0"

# infrared wavelengths
min_wavelength =  780e-3 # micron
max_wavelength = 1000 # micron
output_file_name = "infrared_gas.txt"

In [13]:
print(f" --------------------------- {galaxy_name} ---------------------------")
print(f"min_wavelength: {min_wavelength} um --- max_wavelength: {max_wavelength} um")


time_start_of_the_code = time.time()

############### Reading files
# Define the name of the galaxy and gas particles files.

directory_path = f"/mnt/raid-cita/dtolgay/FIRE/post_processing_fire_outputs/skirt/runs_hden_radius/{which_FIRE}/z{redshift}/{galaxy_name}/trial1"    
# directory_path = f"/mnt/raid-cita/dtolgay/FIRE/post_processing_fire_outputs/skirt/runs_hden_radius/{which_FIRE}/z{redshift}/{galaxy_name}/level5_11_tau2_frac6"    

# directory_path = f"/mnt/raid-cita/dtolgay/FIRE/post_processing_fire_outputs/skirt/runs_hden_radius/{which_FIRE}/z{redshift}/{galaxy_name}/average_sobolev_smoothingLength"    
skirt_galaxy_path_wout_extension = f"{directory_path}/{galaxy_name}"

print(f"directory_path: {directory_path}")

### Check if file exits. If it exists do not runs the code: 
if os.path.isfile(f"{directory_path}/{output_file_name}"):
    print("File exits. Returning nothing!")
    sys.exit(1)
else:
    print(f"{directory_path}/{output_file_name} doesn't exitst. Continuing...")



#### Get the wavelengths 
wavelengths_filename = skirt_galaxy_path_wout_extension +"_grid_radiationField_wavelengths.dat"
print(f"\nwavelengths_filename: {wavelengths_filename}")
wavelength_indices, wavelengths = find_indices_of_wavelengths(wavelengths_filename, 
                                                              min_wavelength,
                                                              max_wavelength)        


#### Read cell properties 
start = time.time()

cell_properties_file_name = skirt_galaxy_path_wout_extension + "_properties_spatialCell_cellprops.dat"
print(f"\nStarting to read {cell_properties_file_name} file.")

cell_content = np.loadtxt(cell_properties_file_name, delimiter = ' ', skiprows = 1)

print(f"Read {cell_properties_file_name} file.")

end = time.time()
time_took_reading_cell_properties_file = end - start
print(f"Took {round(time_took_reading_cell_properties_file, 2)} seconds")
####

#### Import FIRE gas data
gas_particles_file_name = "comprehensive_gas.txt"      
gas_particles_path = f"{directory_path}/{gas_particles_file_name}"
print(f"\nStarting to read {gas_particles_path} file.")

gas_particles = np.loadtxt(gas_particles_path, skiprows = 1)
print(f"Read {gas_particles_path} file.")  
####

#### Read the radiationField_J.dat file 
start = time.time()

wavelength_indices_in_radiationField_J_file = wavelength_indices + 1 
indices = np.append(0, wavelength_indices_in_radiationField_J_file)  # Add zero to get the cell index 

radiationField_J_filename = skirt_galaxy_path_wout_extension + "_grid_radiationField_J.dat"
print(f"\nI am reading the {radiationField_J_filename} file")
radiationField_content = np.loadtxt(radiationField_J_filename, delimiter=' ', skiprows=1, usecols = indices)  # Only the habing radiation field indices are imported

end = time.time()
time_took_reading_radiationField_J_dat = end - start
print(f"{radiationField_J_filename} read! Took {round(time_took_reading_radiationField_J_dat / 60, 2) } minutes")
#### 

 --------------------------- gal200 ---------------------------
min_wavelength: 0.78 um --- max_wavelength: 1000 um
directory_path: /mnt/raid-cita/dtolgay/FIRE/post_processing_fire_outputs/skirt/runs_hden_radius/firebox/z0.0/gal200/trial1
/mnt/raid-cita/dtolgay/FIRE/post_processing_fire_outputs/skirt/runs_hden_radius/firebox/z0.0/gal200/trial1/infrared_gas.txt doesn't exitst. Continuing...

wavelengths_filename: /mnt/raid-cita/dtolgay/FIRE/post_processing_fire_outputs/skirt/runs_hden_radius/firebox/z0.0/gal200/trial1/gal200_grid_radiationField_wavelengths.dat
I am in the function find_indices_of_wavelengths
/mnt/raid-cita/dtolgay/FIRE/post_processing_fire_outputs/skirt/runs_hden_radius/firebox/z0.0/gal200/trial1/gal200_grid_radiationField_wavelengths.dat is started to read.
/mnt/raid-cita/dtolgay/FIRE/post_processing_fire_outputs/skirt/runs_hden_radius/firebox/z0.0/gal200/trial1/gal200_grid_radiationField_wavelengths.dat was read.

Starting to read /mnt/raid-cita/dtolgay/FIRE/post_proc

In [52]:
radiationField_content[0]

array([0.00000000e+00, 8.46498569e-09, 1.85073137e-08, 8.36400831e-09,
       1.09982352e-08, 7.75923117e-09, 9.82525038e-09, 1.00698983e-08,
       8.82907222e-09, 6.28001241e-09, 5.94654047e-09, 1.35414180e-08,
       8.43572867e-09, 6.13654408e-09, 6.48926268e-09, 7.88239943e-09,
       1.12284379e-08, 6.75885395e-09, 7.86009175e-09, 1.07415225e-08,
       6.69286899e-09, 4.62893758e-09, 4.57513478e-09, 4.97342260e-09,
       9.48229587e-09, 4.21074145e-09, 2.38881874e-09, 3.79749953e-09,
       4.83725613e-09, 3.26804291e-09, 3.12058579e-09, 4.18861781e-09,
       1.24024511e-09, 2.66238555e-09, 2.68696154e-09, 1.95379830e-09,
       1.44705933e-09, 1.15813278e-09, 7.32137597e-10, 9.51071746e-10,
       1.14277805e-09, 9.38482877e-10, 6.58557162e-10, 3.27098426e-10,
       7.87463603e-10, 7.82814642e-10, 5.45445015e-10, 6.42686914e-10,
       2.21856455e-10, 8.57573735e-10, 2.05954138e-10, 5.81566512e-10,
       6.26841703e-10, 1.39636872e-10, 2.66145983e-10, 2.28568841e-10,
      

In [80]:
%%time
flux = []
# for cell_radiation_field in radiationField_content[0:int(1e4)]: # TODO: Testing purposses
for i, cell_radiation_field in enumerate(radiationField_content): 
    flux.append(
        flux_calculator(cell_radiation_field, wavelengths)
    )
    if (i % 1e5 == 1):
        print(f"{i} finished. Left {len(radiationField_content) - i}")

# Convert it to a numpy array
flux = np.array(flux)

1 finished. Left 9602621
100001 finished. Left 9502621
200001 finished. Left 9402621
300001 finished. Left 9302621
400001 finished. Left 9202621
500001 finished. Left 9102621
600001 finished. Left 9002621
700001 finished. Left 8902621
800001 finished. Left 8802621
900001 finished. Left 8702621
1000001 finished. Left 8602621
1100001 finished. Left 8502621
1200001 finished. Left 8402621
1300001 finished. Left 8302621
1400001 finished. Left 8202621
1500001 finished. Left 8102621
1600001 finished. Left 8002621
1700001 finished. Left 7902621
1800001 finished. Left 7802621
1900001 finished. Left 7702621
2000001 finished. Left 7602621
2100001 finished. Left 7502621
2200001 finished. Left 7402621
2300001 finished. Left 7302621
2400001 finished. Left 7202621
2500001 finished. Left 7102621
2600001 finished. Left 7002621
2700001 finished. Left 6902621
2800001 finished. Left 6802621
2900001 finished. Left 6702621
3000001 finished. Left 6602621
3100001 finished. Left 6502621
3200001 finished. Left 

In [85]:
# Write to a file 

# Create a dataframe to write to a file 
cell_flux = pd.DataFrame()
cell_flux["index"] = cell_content[:,0]
cell_flux["flux"] = flux


############### Writing to a file 

write_file_path = f"{directory_path}/{output_file_name}"

header = f"""Flux for {galaxy_name} galaxy between wavelengths {min_wavelength} um and {max_wavelength} um.
Column 0: gas_index (1)
Column 1: Flux (W/m^2)
"""

print(f"Started to write to {write_file_path}")
np.savetxt(fname=write_file_path, X=cell_flux, fmt='%.4e', header=header)
print(f"File written to {write_file_path}")


Started to write to /mnt/raid-cita/dtolgay/FIRE/post_processing_fire_outputs/skirt/runs_hden_radius/firebox/z0.0/gal200/trial1/infrared_gas.txt
File written to /mnt/raid-cita/dtolgay/FIRE/post_processing_fire_outputs/skirt/runs_hden_radius/firebox/z0.0/gal200/trial1/infrared_gas.txt


,index,flux
0,0.0,1.393078e-07
1,1.0,1.462548e-07
2,2.0,1.337936e-07
3,3.0,1.428213e-07
4,4.0,1.453064e-07
...,...,...
9602617,9602617.0,9.516564e-06
9602618,9602618.0,1.019875e-05
9602619,9602619.0,1.558505e-05
9602620,9602620.0,7.729468e-06
